## Masking

有时我们想要在计算管道表达式时忽略某些资产。忽略资产有两种常见情况是有用的：
1. 我们想计算一个计算成本很高的表达式，我们知道我们只关心某些资产的结果。这种昂贵的表达式的常见例子是计算回归系数的因子（`RollingLinearRegressionOfReturns`）。
2. 我们想要计算一个表达式来执行资产之间的比较，但我们只希望针对所有资产的子集执行这些比较。例如，我们可能想使用因子的`top`方法来计算收益率最高的200个资产，忽略不符合某些流动性约束的资产。

为了支持这两种用例，所有因子和许多因子方法都可以接受一个掩码参数，该参数必须是一个`Filter`，用于指示计算时要考虑的资产。

## Masking Factors

比方说，我们希望我们的管道能够输出具有高或低百分比差异的证券，但我们也只想考虑价值超过10,000,000美元的证券。为此，我们重新安排我们的`make_pipeline`函数，以便我们首先创建`high_dollar_volume`过滤器。然后，我们可以使用此过滤器作为移动平均因子的掩码，方法是将`high_dollar_volume`作为掩码参数传递给`SimpleMovingAverage`。

In [1]:
from zipline.pipeline.factors import AverageDollarVolume, SimpleMovingAverage
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.research import run_pipeline

In [2]:
# Dollar volume factor
dollar_volume = AverageDollarVolume(
    window_length=30,
    inputs=[USEquityPricing.close, USEquityPricing.volume])

# High dollar volume filter
high_dollar_volume = (dollar_volume > 10000000)

# Average close price factors
mean_close_10 = SimpleMovingAverage(
    inputs=[USEquityPricing.close],
    window_length=10,
    mask=high_dollar_volume
)
mean_close_30 = SimpleMovingAverage(
    inputs=[USEquityPricing.close],
    window_length=30,
    mask=high_dollar_volume
)

# Relative difference factor
percent_difference = (mean_close_10 - mean_close_30) / mean_close_30

将掩码应用于`SimpleMovingAverage`将平均收盘价格因子限制在通过`high_dollar_volume`过滤器的约2000只证券的计算上，而不是没有掩码的约8000只证券。当我们将`mean_close_10`和`mean_close_30`组合形成`percent_difference`时，计算在相同的约2000只证券上执行。

## Masking Filters

Masks也可以应用于返回过滤器的`top`，`bottom`和`percentile_between`之类方法上。

当我们想要在组合计算的早期步骤中应用过滤时，Masks是最有用的。例如，假设我们想要获得最高开盘价的50种证券，这些证券也是美元数量的前10%。假设我们然后希望通过收盘价获得这些证券的90-100百分位数。我们可以通过以下方式做到这一点：

In [3]:
# Dollar volume factor
dollar_volume = AverageDollarVolume(
    window_length=30,
    inputs=[USEquityPricing.close, USEquityPricing.volume])

# High dollar volume filter
high_dollar_volume = dollar_volume.percentile_between(90,100)

# Top open price filter (high dollar volume securities)
top_open_price = USEquityPricing.open.latest.top(50, mask=high_dollar_volume)

# Top percentile close price filter (high dollar volume, top 50 open price)
high_close_price = USEquityPricing.close.latest.percentile_between(90, 100, mask=top_open_price)

我们把它放到`make_pipeline`中，并输出一个用我们的`high_close_price`筛选器筛选出的空管道。

In [5]:
def make_pipeline():

    # Dollar volume factor
    dollar_volume = AverageDollarVolume(
        window_length=30,
        inputs=[USEquityPricing.close, USEquityPricing.volume])

    # High dollar volume filter
    high_dollar_volume = dollar_volume.percentile_between(90, 100)

    # Top open securities filter (high dollar volume securities)
    top_open_price = USEquityPricing.open.latest.top(
        50, mask=high_dollar_volume)

    # Top percentile close price filter (high dollar volume, top 50 open price)
    high_close_price = USEquityPricing.close.latest.percentile_between(
        90, 100, mask=top_open_price)

    return Pipeline(screen=high_close_price)

运行这条管道，2015年5月5日产出约5个证券。

In [6]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
print('Number of securities that passed the filter: %d' % len(result))

/home/ldf/source/czipline/zipline/pipeline/filters/filter.py:380: RuntimeWarning: invalid value encountered in less_equal
  return (lower_bounds <= data) & (data <= upper_bounds)


Number of securities that passed the filter: 5


请注意，如上所述应用图层中的蒙版可以被认为是“资产漏斗”。

在下一课中，我们将看看分类器。